In [11]:
import numpy as np
import pandas as pd
import datetime
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from scipy import stats
from math import sqrt
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]
%matplotlib inline

In [2]:
marag_df = pd.read_csv('./../../../Databases/marag_lags_data.csv')
marag_df.set_index('Timestamp', inplace=True)
marag_df.sort_index(inplace=True)

# For the time being we will not use the outliers column, so we will drop it
marag_df.drop(['outliers'], axis=1, inplace=True)

In [3]:
# Let's import the marag data with no lags for future comparative purposes
no_lags_marag = pd.read_csv('./../../../Databases/marag_data.csv')
no_lags_marag.set_index('Timestamp', inplace=True)
no_lags_marag.sort_index(inplace=True)
no_lags_marag.drop(['outliers'], axis=1, inplace=True)

In [4]:
from ipynb.fs.full.train_dev_test_split import train_dev_test_split

X_train, X_dev, X_test, y_train, y_dev, y_test = train_dev_test_split(marag_df.drop(['TotalEntries'], axis=1), 
                                                                      marag_df['TotalEntries'])

lr_X_train = pd.concat([X_train, X_dev])
lr_X_train.sort_index(inplace=True)
lr_y_train = pd.concat([y_train, y_dev])
lr_y_train.sort_index(inplace=True)

In [5]:
lr_model = LinearRegression().fit(lr_X_train, lr_y_train)
lr_y_hat = lr_model.predict(X_test)
lr_y_hat = pd.Series(lr_y_hat, index=y_test.index, name='Predicted_TotalEntries')

In [14]:
from ipynb.fs.full.get_all_predictions import get_all_predictions
import time
start = time.time()

all_future_df = get_all_predictions(no_lags_marag, X_train, X_test, lr_model)

end = time.time()
print(end - start)

100%|██████████| 2/2 [00:02<00:00,  1.15s/it]

2.3840224742889404


In [ ]:
all_future_df.columns = ['TotEntr_PUF_allcols']

In [9]:
all_future_df.to_csv('./../../../Databases/TotEntre_PUF_alcols.csv', index=True)